# Hamiltonian simulation with Gray code encoding - statevector simulator

In [89]:
import numpy as np
np.warnings.filterwarnings('ignore')

import pickle

from scipy.linalg import expm
import scipy
from pprint import pprint
from tqdm import tqdm

# Everything we need from Qiskit
from qiskit import ClassicalRegister, QuantumRegister, QuantumCircuit
from qiskit import execute, Aer
from qiskit.quantum_info import Pauli

from qiskit.aqua.operators import WeightedPauliOperator
from qiskit.aqua.components.initial_states import Custom

import qutip as qt

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set(rc={'figure.figsize':(16,8)})
sns.set()

import itertools

import sys
sys.path.append("../src/")
from hamiltonian import *
from utils import *
from qiskit_circuits import *

## Playing with Pauli and WeightedPauli operators

In [2]:
import itertools

## H is always in qiskit order (right to left)
## String form is also in that order 

## State vector results in order [00,01,10,11]

n_qubits=2
H = DenseEncodingHamiltonian(N_states=2**n_qubits,qiskit_order=False)
print(H.pauli_coeffs)
# get_pauli_matrix("IXX")

# gc_ham_rep = reduce(lambda x, y: x + y, [p[1] * get_pauli_matrix(p[0]) for p in H.pauli_coeffs.items()])
# print(gc_ham_rep)


## Get all possible pauli operators for given number of qubits 
pauli_strings=["I","Z","X","Y"]
pauli_product_strings=list(map(lambda x : "".join(x),list(itertools.product(*[pauli_strings for i in range(n_qubits)]))))

## Extract out pauli terms of odd number of Y 
paulis=[]
for string in pauli_product_strings:
    if string.count("Y")%2==1:
            paulis.append(Pauli.from_label(string))
            
# for pauli in paulis:
#     print(pauli)

sigmas=[]
for pauli in paulis:
    sigma=[(1.0,pauli)]
    sigmas.append(WeightedPauliOperator(sigma))




## Generate intial guess for A (NO)
a=np.random.uniform(0,1,len(paulis))
pairs=list(zip(a,paulis))
weighted_pauli_op=WeightedPauliOperator(pairs)    
# print(weighted_pauli_op.print_details())



# commuting_sets={}
# idx=0
# for wp1 in w_paulis:
#     for wp2 in w_paulis:
#         print(wp1.commute_with(wp2))

print(pauli_product_strings)
# {'IX': -7.813951636042159, 'ZX': 3.5273445861715977, 'XI': -3.913118960624632, 'XZ': 3.913118960624632, 'II': 14.3283547225, 'ZI': -8.4216452775, 'IZ': -1.4216452774999997, 'ZZ': -4.9216452775}

{'IX': -7.813951636042159, 'ZX': 3.5273445861715977, 'XI': -3.913118960624632, 'XZ': 3.913118960624632, 'II': 14.3283547225, 'ZI': -8.4216452775, 'IZ': -1.4216452774999997, 'ZZ': -4.9216452775}
['II', 'IZ', 'IX', 'IY', 'ZI', 'ZZ', 'ZX', 'ZY', 'XI', 'XZ', 'XX', 'XY', 'YI', 'YZ', 'YX', 'YY']


In [76]:
def commutes(a,b):
    """
    Returns true if two operators commute
    
    input:
        a,b (str) : Pauli operators.  Must be "X", "Y", "Z", "I"
    """
    #Todo add raise error if a[i] not in allowed pauli set
    does_commute=True
    for i in range(len(a)):
        ## If ai or bi are identity then ith paulis will commute 
        if a[i]=='I' or b[i]=='I':
            continue
        
        ## if ai != bi then they won't commute 
        if a[i]!=b[i]:
            does_commute=False
            break

    return does_commute

def get_commuting_sets(paulis):
    """
    Get a dictionary of commuting sets.  Key for each set
    is first term of the set
    
    input:
        paulis (list<str>) : list of pauli terms
    
    """
    ## Initalize set with identity string
    n_qubits=len(paulis[0])
    identity="I"*n_qubits
    
    commuting_sets={}

    for p1 in paulis:
        ## Identity should not be commuting set key since everything commutes 
        if p1 == identity:
            continue

        ## If no commuting set, start one. 
        if commuting_sets=={}:
            commuting_sets[p1]=[p1]
            
        ## iterate through keys in commuting set and check if operator commutes    
        for p2 in commuting_sets:
            does_commute=commutes(p1,p2)

            ## operator commutes with identifier of commuting set, check that it 
            ## commutes with every other term in commuting set
            if does_commute:                
                
                for p3 in commuting_sets[p2]:
                    if not commutes(p1,p3):
                        does_commute=False
                        break
                
            ## if operator commuted with all operators in commuting set, add to set   
            if does_commute:
                commuting_sets[p2].append(p1)
                break
            
        ## if it didn't commute with any, create new commuting set 
        if not does_commute:
            commuting_sets[p1]=[p1]


    ## Add identity to last commuting set (probably has fewest terms)
    commuting_sets[list(commuting_sets.keys())[-1]].append(identity)
    
    return commuting_sets





def get_pauli_terms(weighted_pauli):
    """
    Extract out individual paul terms 
    
    Input:
        weighted_pauli ():
        
    Return
        terms (dictionary) : dictionary of {label:coef} for each pauli term. 
                label is pauli string
    """
    terms={}
    for term in weighted_pauli.to_dict()['paulis']:
        label=term['label']
        coef=term['coeff']['real']+1j*term['coeff']['imag']
        terms[label]=coef
    return terms

def get_sigma_pauli_terms(n_qubits):
    """Get all possible pauli operators for given number of qubits """ 
    pauli_strings=["I","X","Y","Z"]
    pauli_product_strings=list(map(lambda x : "".join(x),list(itertools.product(*[pauli_strings for i in range(n_qubits)]))))

    ## Extract out pauli terms of odd number of Y 
    paulis=[]
    for string in pauli_product_strings:
        if string.count("Y")%2==1:
            paulis.append(Pauli.from_label(string))

    sigmas=[]
    for pauli in paulis:
        sigma=[(1.0,pauli)]
        sigmas.append(WeightedPauliOperator(sigma))

    return sigmas


def H_weighted_paulis(H):
    H_pairs=[(v, Pauli.from_label(k)) for (k, v) in list(H.pauli_coeffs.items())]
    return WeightedPauliOperator(H_pairs)
    
# def c_coef(H):
# #     coef=1-2*t<H>
    
    
def b_terms(H,sigmas):
    b_pauli_terms=[]
    H_paulis=H_weighted_paulis(H)
    for sigma in sigmas:
        product=1j*(H_paulis.__mul__(sigma)-sigma.__mul__(H_paulis))
        product.chop(1e-5)
        terms=get_pauli_terms(product)
        b_pauli_terms.append(terms)
    return b_pauli_terms

## define S array
def S_terms(sigmas):
    d=len(sigmas)
    S={}
    for i in range(d):
        sigma1=sigmas[i]
        for j in range(d):
            sigma2=sigmas[j]
            product=sigma1.__mul__(sigma2)
            S[(i,j)]=get_pauli_terms(product)
    return S


## Get intersection of pauli terms needed to compute S and b
def get_intersection_pauli_terms(H,b_pauli_terms,S_pauli_terms):
    ## Initialize set with pauli terms in Hamiltonian
    pauli_set=set(H.pauli_coeffs.keys())
    
    ## Add b 
    for bI in b_pauli_terms:
        pauli_set.update(bI.keys())

    ## Add S
    for term in S_pauli_terms.values():
            pauli_set.update(term.keys())

    return pauli_set



def initialize_circuit(q,initial_state=None):
    """Initialize circuit.  If initial state not given, start with uniform superposition."""

    if initial_state==None:
        circuit = QuantumCircuit(q)
#         circuit.h(q[1])
    else:
        #Initilize with initial state
        circuit = initial_state.construct_circuit('circuit', q)
    return circuit

def get_evolution_circuit(q,A_set,circuit):
    """
    Append evolution exp(-iAt) onto circuit for each time step t
    
    input:
        q (qiskit.circuit.quantumregister.QuantumRegister) : qubits
        A_set () : List of Weighted Pauli operators by which the circuit is evolved at each timestep
        circuit (qiskit.circuit.quantumcircuit.QuantumCircuit) : quantum circuit 

    return
        circuit (qiskit.circuit.quantumcircuit.QuantumCircuit) : updated quantum circuit
    """

    for A in A_set: 
        #Append next step to circuit for each A
        circuit += A.evolve(
            None, evo_time=1, num_time_slices=1,
            quantum_registers=q
            )
    return circuit  

def get_measurement_circuit(q,circuit,set_id):
    """
    Append gates to transform qubit states to measurement basis 
    
    input 
        q (qiskit.circuit.quantumregister.QuantumRegister) : qubits
        circuit (qiskit.circuit.quantumcircuit.QuantumCircuit) : quantum circuit 
        set_id (str) : pauli operator to be measured
    return
        circuit (qiskit.circuit.quantumcircuit.QuantumCircuit) : updated quantum circuit        
    """
    ## Reverse order to gates are applied right to left
    id=set_id[::-1]
    
    for qubit_idx in range(len(set_id)):
        pauli=id[qubit_idx]
        if pauli == 'X':
            circuit.h(q[qubit_idx])
        elif pauli == 'Y':
            circuit.sdg(q[qubit_idx])
            circuit.h(q[qubit_idx])
    return circuit


def compute_expectation_value(pauli,meas_results):
    pauli_list = list(pauli)
    n_qubits = len(pauli_list)
    expectation_value=0.0
    for basis_state in meas_results.keys():
        num_z_and_1 = [-1 if (basis_state[bit_idx] == '1' and pauli_list[bit_idx] != 'I') else 1 for bit_idx in range(n_qubits)]
        eigenvalue = reduce(lambda x, y: x*y, num_z_and_1)
        expectation_value+=eigenvalue*meas_results[basis_state]

    return expectation_value

def get_commuting_sets2(paulis):
    """
    Get a dictionary of commuting sets.  Key for each set
    is first term of the set
    
    input:
        paulis (list<str>) : list of pauli terms
    
    """
    
    commuting_array=[]
    paulis=list(paulis)
    for p1 in paulis:
        does_commute=False
        for i in range(len(commuting_array)):
            p_set=commuting_array[i]
            for p2 in p_set:
                does_commute=commutes(p1,p2)
                if not does_commute:
                    break
            if does_commute:
                commuting_array[i].append(p1)
                break
        if not does_commute:
            commuting_array.append([p1])

    ## identify pauli which will define measurement basis
    commuting_sets={}
    n_qubits=len(paulis[0])
    for p_set in commuting_array:
        minI=n_qubits
        identitfier=None
        for p in p_set:
            numI=p.count("I")
            if numI < minI:
                minI=numI
                identifier=p
        commuting_sets[identifier]=p_set
    return commuting_sets

get_commuting_sets2(pauli_set)


{'YZ': ['YZ', 'IZ', 'YI', 'II'],
 'XX': ['XX', 'IX', 'XI'],
 'ZY': ['IY', 'ZY', 'ZI'],
 'ZX': ['ZX'],
 'ZZ': ['ZZ'],
 'XZ': ['XZ'],
 'XY': ['XY'],
 'YY': ['YY'],
 'YX': ['YX']}

In [77]:
n_qubits=2
H = DenseEncodingHamiltonian(N_states=2**n_qubits,qiskit_order=False)
print(H.pauli_coeffs)



{'IX': -7.813951636042159, 'ZX': 3.5273445861715977, 'XI': -3.913118960624632, 'XZ': 3.913118960624632, 'II': 14.3283547225, 'ZI': -8.4216452775, 'IZ': -1.4216452774999997, 'ZZ': -4.9216452775}


In [78]:
## vector of WeightedPauliOperators representing vector Sigma

## Get list of sigmas (all pauli terms with odd number Y gates)
sigmas=get_sigma_pauli_terms(n_qubits)

## Construct b
b_pauli_terms=b_terms(H,sigmas)
# for bI in b_pauli_terms:
#     print(bI)
      
## Construct S
S_pauli_terms=S_terms(sigmas)

pauli_set=get_intersection_pauli_terms(H,b_pauli_terms,S_pauli_terms)

## Sorting forces II to be one of the identifiers
## and first in the commuting sets
commuting_sets=get_commuting_sets2(sorted(pauli_set))
for p in commuting_sets:
    print(p, commuting_sets[p],p[::-1])
    


XX ['II', 'IX', 'XI', 'XX'] XX
XY ['IY', 'XY'] YX
XZ ['IZ', 'XZ'] ZX
YX ['YI', 'YX'] XY
YY ['YY'] YY
YZ ['YZ'] ZY
ZX ['ZI', 'ZX'] XZ
ZY ['ZY'] YZ
ZZ ['ZZ'] ZZ


In [104]:
from scipy.linalg import lstsq
import scipy as sp
#####################################       
# Prepare and run the evolution circuit
#####################################       

print(commuting_sets)
verbose=False
backend = Aer.get_backend('statevector_simulator')
q = QuantumRegister(n_qubits)
c = ClassicalRegister(n_qubits)
## When t=0, initilize circuit with None option for circuit
## Start with linear combination of all states with same coefficient

#psi = 0.5 * np.array([[1], [1], [1], [1]])

a=np.zeros(len(sigmas))
A_set=[]
num_iterations=10
delta_time=0.05
time=delta_time*num_iterations
Energies=np.zeros(num_iterations)
## for each time step, run circuit and compute A for the next time step
for t in range(num_iterations):
    ## Initialize circuit

    counts_dict={}    
    
    for pauli_id in commuting_sets:    
        ## If not t=0, then evolve cirucit using previously computed A matrices stored in A_set
        circuit=initialize_circuit(q,initial_state=None)
        
        if t>0:
            circuit=get_evolution_circuit(q,A_set,circuit)
        
        ## Get circuit measurement 
        circuit=get_measurement_circuit(q,circuit,pauli_id)

        job = execute(circuit, backend)
#         print(circuit.draw())
#         print(pauli_id)
#         print(job.result().get_counts(circuit))
        counts_dict[pauli_id] = job.result().get_counts(circuit)
        if pauli_id=='ZZ':
            psi=job.result().get_statevector(circuit)
#             print(psi)
#             print(counts_dict[pauli_id])
#         print(job.result().get_counts(circuit))
#         if pauli_id == "ZZ":
#         print("step ",t)
#         print(psi)
# #         print(circuit.draw())
#         print("--------------------------------")
# #         print(pauli_id)
# #         print(wavefunction)
        

    expectation_values={}
    expectation_values2={}
    for pauli_id in commuting_sets:    
        #Comput expectation value for each pauli term 
        meas_results=counts_dict[pauli_id]
        for pauli in commuting_sets[pauli_id]: 

            ## Need to flip order to get matrix constructed right to left to match vector??
            pauli_mat = get_pauli_matrix(pauli)

            # Temp
            e_value=np.conj(psi).T @ pauli_mat @ psi
            e_value2=compute_expectation_value(pauli,meas_results).astype(complex)
            expectation_values[pauli]=e_value
            expectation_values2[pauli]=e_value2
#             print(type(e_value),type(e_value2))
            if abs(e_value2-e_value)>1e-10:
                print(pauli_id,pauli,e_value, e_value2)
            
#             print(type(expectation_values2[pauli]),type(expectation_values[pauli]))
    
    ## Compute energy

    H_pauli=H.pauli_coeffs
    for key in H_pauli:
        Energies[t]+=H_pauli[key]*expectation_values[key]
#     H_paulis=H_weighted_paulis(H)
#     print(H.pauli_coeffs)
    
    ## compute normalization coef C=1-2*E*delta_times
    ## 1/sqrt(C) approx 1-E*delta_time
    Ccoef=1-delta_time*Energies[t]
#     print(Energies[t],delta_time,Ccoef)
#     print("Ccoef",Ccoef)
    
    ## compute b
    num_sigmas=len(sigmas)
    b=np.zeros(num_sigmas)
    for sigma_idx in range(num_sigmas):
        bI=b_pauli_terms[sigma_idx]
        for term in bI:
            b[sigma_idx]+=bI[term]*expectation_values[term]

    b=b/Ccoef
#     print(b)       
    
    ## Comput S
    Smatrix=np.asmatrix(np.zeros((num_sigmas,num_sigmas)))
#     print(type(Smatrix))
    for i in range(num_sigmas):
        for j in range(num_sigmas):
            Sij=S_pauli_terms[(i,j)]
            for term in Sij:
                Smatrix[i,j]+=2*Sij[term]*expectation_values[term] #(S+S^T) gives rise to factor of 2


    a,_,_,_=lstsq(Smatrix,b)
    if verbose==True:
        print("Smatrix")
        print(Smatrix)
        print("b\n",b)


        print("a")
        print(a)
#         print("Sa-b")
#         print(Smatrix.dot(a)-b)
    
    ## Construct A from a
    identity_string="I"*n_qubits

    ##Null initialize A with 0*I. 
    A=WeightedPauliOperator([(0.0,Pauli.from_label(identity_string))])
#     print("num sigmas ",len(sigmas))
    for i in range((len(sigmas))):
        A+=delta_time*a[i]*sigmas[i]
#         print(delta_time*a[i],sigmas[i].print_details())
    if verbose==True:       
        print("t=",t)
        print("-----------------------------------------")

        print("wavefunction\n",wavefunction)        
        print("\nA operator")
        print(A.print_details())
        
        A_pauli_terms=get_pauli_terms(A)
        Amatrix=reduce(lambda x,y: x+y,[A_pauli_terms[term]*get_pauli_matrix(term) for term in A_pauli_terms])
#         print(Amatrix)
        print("-----------------------------------------")
    A_set.append(A)
    
## Compute H   

print("Energies")
print(Energies)


{'XX': ['II', 'IX', 'XI', 'XX'], 'XY': ['IY', 'XY'], 'XZ': ['IZ', 'XZ'], 'YX': ['YI', 'YX'], 'YY': ['YY'], 'YZ': ['YZ'], 'ZX': ['ZI', 'ZX'], 'ZY': ['ZY'], 'ZZ': ['ZZ']}
Energies
[-0.43658111 -1.63252759 -1.93192748 -2.05914552 -2.10964378 -2.12261428
 -2.1349186  -2.14013326 -2.14231486 -2.14325341]


In [7]:
def unitary_evolution(A, t):
    return expm(-1j * A* t)

A_pauli_terms=get_pauli_terms(A)
for p in A_pauli_terms:
    print( p, A_pauli_terms[p])

# pp=[p[1] * get_pauli_matrix(p[0]) for p in H.pauli_coeffs.items()]
Amatrix=reduce(lambda x,y: x+y,[A_pauli_terms[term]*get_pauli_matrix(term) for term in A_pauli_terms])
print(Amatrix)
    
uniform_gc = 0.5 * np.array([[1], [1], [1], [1]])
wavefunction = unitary_evolution(Amatrix, t=time) @ uniform_gc
true_probabilities = (wavefunction * np.conj(wavefunction)).flatten()
print(wavefunction)
print(true_probabilities)

IY (0.10487583580633858+0j)
XY 0j
YI 0j
YX 0j
YZ 0j
ZY (0.10487583580633855+0j)
[[0.+0.00000000e+00j 0.-2.09751672e-01j 0.+0.00000000e+00j
  0.+0.00000000e+00j]
 [0.+2.09751672e-01j 0.+0.00000000e+00j 0.+0.00000000e+00j
  0.+0.00000000e+00j]
 [0.+0.00000000e+00j 0.+0.00000000e+00j 0.+0.00000000e+00j
  0.-2.77555756e-17j]
 [0.+0.00000000e+00j 0.+0.00000000e+00j 0.+2.77555756e-17j
  0.+0.00000000e+00j]]
[[0.49472881+0.j]
 [0.5052162 +0.j]
 [0.5       +0.j]
 [0.5       +0.j]]
[0.24475659+0.j 0.25524341+0.j 0.25      +0.j 0.25      +0.j]
